In [1]:
import numpy as np 
import pandas as pd 
#import plotly.graph_objs as go
#from plotly.offline import init_notebook_mode, iplot
#init_notebook_mode(connected=True)
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns 
color = sns.color_palette()
#from pandas.tools.plotting import parallel_coordinates
from sklearn.decomposition import PCA
from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier

In [2]:
train = pd.read_csv("C:/Users/Mrinal/Desktop/Forest/train.csv") # store as dataframe objects
test = pd.read_csv("C:/Users/Mrinal/Desktop/Forest/test.csv") # store as dataframe objects

In [3]:
y=train['Cover_Type']
test_id=test['Id']
train.drop('Id', axis=1, inplace=True)
test.drop('Id', axis=1, inplace=True)


In [4]:
print('Train shape: {}'.format(train.shape))
print('Test  shape: {}'.format(test.shape))

Train shape: (15120, 55)
Test  shape: (565892, 54)


In [5]:
train.columns

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40

In [6]:
def features(data):
    data['HHydro_Fire'] = (data['Horizontal_Distance_To_Hydrology']+data['Horizontal_Distance_To_Fire_Points'])
    data['Neg_HHydro_Fire'] = (data['Horizontal_Distance_To_Hydrology']-data['Horizontal_Distance_To_Fire_Points'])
    data['Mean_HHydro_Fire'] = (data['Horizontal_Distance_To_Hydrology']+data['Horizontal_Distance_To_Fire_Points'])/2
    data['Mean_Neg_HHydro_Fire'] = (data['Horizontal_Distance_To_Hydrology']-data['Horizontal_Distance_To_Fire_Points'])/2
    data['Mean_Fire_HHydro-rev']=(data['Horizontal_Distance_To_Fire_Points'] + data['Horizontal_Distance_To_Hydrology']) / 2
    
    data['HHydro_Road'] = (data['Horizontal_Distance_To_Hydrology']+data['Horizontal_Distance_To_Roadways'])
    data['Neg_HHydro_Road'] = (data['Horizontal_Distance_To_Hydrology']-data['Horizontal_Distance_To_Roadways'])
    data['Mean_HHydro_Road'] = (data['Horizontal_Distance_To_Hydrology']+data['Horizontal_Distance_To_Roadways'])/2
    data['Mean_Neg_HHydro_Road'] = (data['Horizontal_Distance_To_Hydrology']-data['Horizontal_Distance_To_Roadways'])/2

    data['Fire_Road'] = (data['Horizontal_Distance_To_Fire_Points']+data['Horizontal_Distance_To_Roadways'])
    data['Neg_Fire_Road'] = (data['Horizontal_Distance_To_Fire_Points']-data['Horizontal_Distance_To_Roadways'])
    data['Mean_Fire_Road'] = (data['Horizontal_Distance_To_Hydrology']+data['Horizontal_Distance_To_Roadways'])/2
    data['Mean_Neg_Fire_Road'] = (data['Horizontal_Distance_To_Fire_Points']-data['Horizontal_Distance_To_Roadways'])/2

    data['Elevation_V'] = data['Elevation']+data['Vertical_Distance_To_Hydrology']
    data['Neg_Elevation_V'] = data['Elevation']-data['Vertical_Distance_To_Hydrology']

    data['Mean_Hillshade'] =  (data['Hillshade_9am']  + data['Hillshade_Noon'] + data['Hillshade_3pm'] ) / 3

    data['Slope_Pyth'] = np.sqrt(data['Horizontal_Distance_To_Hydrology']**2+data['Vertical_Distance_To_Hydrology']**2)
    data['Mean_Fire_Hydro_Road']=(data['Horizontal_Distance_To_Fire_Points'] + data['Horizontal_Distance_To_Hydrology'] + data['Horizontal_Distance_To_Roadways']) / 3

    data["Abs_VHydro"] = abs(data['Vertical_Distance_To_Hydrology'])

    data['Neg_Elevation_HHydro'] = data.Elevation-data.Horizontal_Distance_To_Hydrology*0.2
    data['Neg_Elevation_VHydro'] = data.Elevation-data.Vertical_Distance_To_Hydrology

    return data

In [7]:
train=features(train)

In [8]:
test=features(test)

In [9]:
type(train)
type(test)

pandas.core.frame.DataFrame

In [10]:
train.columns

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40

In [11]:
y_train_T = train['Cover_Type']
X_train=train
X_train.drop(columns = 'Cover_Type', inplace=True)
X_test=test
all_data=pd.concat((train,test)).reset_index(drop=True)

X_train1=X_train
X_test1=X_test
X_train2=X_train
X_test2=X_test

oney=pd.DataFrame(y_train_T)
twoy=pd.DataFrame(y_train_T)



In [12]:
pd.DataFrame(y_train_T).columns

Index(['Cover_Type'], dtype='object')

In [13]:
x=len(y_train_T)
for a in range(0,x):
    if oney.Cover_Type[a] > 1:
        oney.Cover_Type[a]=0

x=len(train)
for a in range(0,x):
    if ((twoy.Cover_Type[a] > 2) | (twoy.Cover_Type[a] == 1)):
        twoy.Cover_Type[a]=0

In [14]:
oney['Cover_Type'].value_counts()

0    12960
1     2160
Name: Cover_Type, dtype: int64

In [15]:
twoy['Cover_Type'].value_counts()

0    12960
2     2160
Name: Cover_Type, dtype: int64

In [16]:
from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier

clf = ExtraTreesClassifier()
clf.fit(X_train,y_train_T)

z = clf.feature_importances_

df = pd.DataFrame()
print(len(z))
print(len(list(X_train.columns.values)))

df["values"] = z
df['column'] = list(X_train.columns.values)
# Sort then descendingly to get the worst features at the end
df.sort_values(by='values', ascending=False, inplace = True)
df.head(100)

75
75


,values,column
68,0.081292,Neg_Elevation_V
74,0.071518,Neg_Elevation_VHydro
67,0.061763,Elevation_V
0,0.057952,Elevation
73,0.047833,Neg_Elevation_HHydro
65,0.028764,Mean_Fire_Road
71,0.027653,Mean_Fire_Hydro_Road
13,0.027007,Wilderness_Area4
23,0.023631,Soil_Type10
10,0.022984,Wilderness_Area1


In [17]:
X_train.shape,X_test.shape, y_train_T.shape

((15120, 75), (565892, 75), (15120,))

In [18]:
df.values=pd.to_numeric(df['values'])
cols=df[df['values']<0.0001].column
cols=np.array(cols)
list(cols)

['Soil_Type9',
 'Soil_Type36',
 'Soil_Type25',
 'Soil_Type15',
 'Soil_Type8',
 'Soil_Type7']

In [19]:
X_train.columns

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40

In [20]:
X_train=X_train.drop(columns=list(cols),  axis = 1 )
X_test=X_test.drop(columns=list(cols),  axis = 1)

In [21]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn import decomposition

scaler = StandardScaler()
scaler.fit(X_train)
x_train = pd.DataFrame(scaler.transform(X_train))
x_test = pd.DataFrame(scaler.transform(X_test))
x_train.columns=X_train.columns
x_test.columns=X_test.columns
y_train=y_train_T

In [22]:
#x_train=X_train
#x_test=X_test
#y_train=y_train_T

In [23]:
#from sklearn.ensemble import ExtraTreesClassifier
#from sklearn.neural_network import MLPClassifier
#from sklearn.model_selection import GridSearchCV
#from sklearn.metrics import classification_report
#from sklearn.ensemble import AdaBoostClassifier
#from sklearn.ensemble import RandomForestClassifier, VotingClassifier
#from sklearn.ensemble import GradientBoostingClassifier

#from xgboost import XGBClassifier

#from sklearn.tree import DecisionTreeClassifier
#from sklearn.metrics import roc_auc_score

#param_grid = {'criterion' : ['entropy', 'gini'], 'n_estimators':[900, 1100, 1300, 1500]}

#tree = GridSearchCV(ExtraTreesClassifier(random_state=0), param_grid, cv=5)

#tree.fit(x_train, y_train)
#tree.best_params_

In [24]:
#clf = ExtraTreesClassifier(n_estimators=900, random_state=0, criterion='entropy')
#clf.fit(x_train, y_train)
#predictions = clf.predict(x_test)
#out = pd.DataFrame()
#out['Id'] = test_id
#out['Cover_Type'] = predictions
#out.to_csv('xxxxx1.csv', index=False)
#out.head(5)

In [25]:
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier(n_estimators=1300, random_state=0, criterion='entropy')
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
#out = pd.DataFrame()
#out['Id'] = test_id
#out['Cover_Type'] = predictions
#out.to_csv('RunForrestRun4.csv', index=False)
#out.head(5)

In [26]:
abcd=pd.DataFrame(predictions)

In [27]:
abcd.head()

,0
0,1
1,1
2,1
3,1
4,1


In [28]:
from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression

clf = ExtraTreesClassifier()
clf.fit(X_train1,oney)

z = clf.feature_importances_

df = pd.DataFrame()
print(len(z))
print(len(list(X_train1.columns.values)))

df["values"] = z
df['column'] = list(X_train1.columns.values)
# Sort then descendingly to get the worst features at the end
df.sort_values(by='values', ascending=False, inplace = True)
df.head(100)

75
75


C:\Users\Mrinal\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


,values,column
68,6.487744e-02,Neg_Elevation_V
0,6.464716e-02,Elevation
74,5.082496e-02,Neg_Elevation_VHydro
67,4.983636e-02,Elevation_V
35,4.932444e-02,Soil_Type22
73,4.821984e-02,Neg_Elevation_HHydro
71,2.821168e-02,Mean_Fire_Hydro_Road
62,2.749442e-02,Mean_Neg_HHydro_Road
36,2.412073e-02,Soil_Type23
58,2.403697e-02,Mean_Fire_HHydro-rev


In [29]:
df.values=pd.to_numeric(df['values'])
cols=df[df['values']<0.0001].column
cols=np.array(cols)
list(cols)

['Soil_Type17',
 'Soil_Type36',
 'Soil_Type3',
 'Soil_Type5',
 'Soil_Type14',
 'Soil_Type8',
 'Soil_Type7',
 'Soil_Type25',
 'Soil_Type1',
 'Soil_Type28',
 'Soil_Type15']

In [30]:
X_train1=X_train1.drop(columns=list(cols),  axis = 1 )
X_test1=X_test1.drop(columns=list(cols),  axis = 1)
y_train1=oney #.drop(columns=list(cols),  axis = 1)


In [31]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn import decomposition

scaler = StandardScaler()
scaler.fit(X_train1)
x_train = pd.DataFrame(scaler.transform(X_train1))
x_test = pd.DataFrame(scaler.transform(X_test1))
x_train.columns=X_train1.columns
x_test.columns=X_test1.columns
y_train=y_train1




clf = ExtraTreesClassifier(n_estimators=1300, random_state=0, criterion='entropy')
clf.fit(x_train, y_train1)
predictions = clf.predict_proba(x_test)
pred = clf.predict(x_test)
p=pd.DataFrame(predictions)
p.columns=['zero','one']





C:\Users\Mrinal\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [32]:
x=len(p)
p['xo']=p['zero']+p['one']
for a in range(0,x):
    if p.zero[a] > 0.7:
        p.xo[a]=0
    else: 
        p.xo[a]=1

In [33]:
one=p['xo']
one.shape

(565892,)

In [34]:
from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression

clf = ExtraTreesClassifier()
clf.fit(X_train,twoy)

z = clf.feature_importances_

df = pd.DataFrame()
print(len(z))
print(len(list(X_train.columns.values)))

df["values"] = z
df['column'] = list(X_train.columns.values)
# Sort then descendingly to get the worst features at the end
df.sort_values(by='values', ascending=False, inplace = True)
df.head(100)

C:\Users\Mrinal\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


69
69


,values,column
67,5.814665e-02,Neg_Elevation_HHydro
68,5.004874e-02,Neg_Elevation_VHydro
0,4.567292e-02,Elevation
62,4.435705e-02,Neg_Elevation_V
10,3.815923e-02,Wilderness_Area1
54,3.223178e-02,Neg_HHydro_Road
22,2.924495e-02,Soil_Type12
61,2.808058e-02,Elevation_V
57,2.774944e-02,Fire_Road
65,2.769308e-02,Mean_Fire_Hydro_Road


In [35]:
df.values=pd.to_numeric(df['values'])
cols=df[df['values']<0.0001].column
cols=np.array(cols)
list(cols)

['Soil_Type35', 'Soil_Type14', 'Soil_Type1', 'Soil_Type5']

In [36]:
X_train2=X_train2.drop(columns=list(cols),  axis = 1 )
X_test2=X_test2.drop(columns=list(cols),  axis = 1)
y_train2=twoy #.drop(columns=list(cols),  axis = 1)


In [37]:
scaler = StandardScaler()
scaler.fit(X_train2)
x_train = pd.DataFrame(scaler.transform(X_train2))
x_test = pd.DataFrame(scaler.transform(X_test2))
x_train.columns=X_train2.columns
x_test.columns=X_test2.columns
y_train=y_train2

clf = ExtraTreesClassifier(n_estimators=1300, random_state=0, criterion='entropy')
clf.fit(x_train, y_train2)
predictions = clf.predict_proba(x_test)
pred = clf.predict(x_test)
p2=pd.DataFrame(predictions)
p2.columns=['zero','two']

C:\Users\Mrinal\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


In [38]:
x=len(p2)
p2['xo']=p2['zero']+p2['two']
for a in range(0,x):
    if p2.zero[a] > 0.7:
        p2.xo[a]=0
    else: 
        p2.xo[a]=2

In [39]:
p.head()

,zero,one,xo
0,0.412308,0.587692,1.0
1,0.311538,0.688462,1.0
2,0.423846,0.576154,1.0
3,0.440000,0.560000,1.0
4,0.433846,0.566154,1.0


In [40]:
p2.head()

,zero,two,xo
0,0.700000,0.300000,2.0
1,0.804615,0.195385,0.0
2,0.694615,0.305385,2.0
3,0.719231,0.280769,0.0
4,0.740000,0.260000,0.0


In [41]:
two=p['xo']
one.shape

(565892,)

In [42]:
out = pd.DataFrame()
out['Id'] = test_id
out['Cover_Typea'] = abcd
out['Cover_Type_one'] = p['xo']
out['Cover_Typea_two'] =p2['xo']
out['d']=0
out.columns=['Id','a','b','c','d']
out.to_csv('azz.csv', index=False)


In [43]:
out.head()

,Id,a,b,c,d
0,15121,1,1.0,2.0,0
1,15122,1,1.0,0.0,0
2,15123,1,1.0,2.0,0
3,15124,1,1.0,0.0,0
4,15125,1,1.0,0.0,0


In [44]:
#x=len(out)
#for i in range(0,x):
#    if ((out.a[i]==1) & (out.b[i]==0) & (out.c[i]==2) ):
#        out.d[i]=2
#    elif ((out.a[i]==2) & (out.b[i]==1) & (out.c[i]==0)) :
#        out.d[i]=2
#    else : out.d[i]=out.a[i]

In [47]:
#out = pd.DataFrame()
#out['Id'] = a['Id']
#out['Cover_Type'] = a['d']
#out.to_csv('Forestofmylife.csv', index=False)
#out.head(5)

,Id,Cover_Type
0,15121,1
1,15122,1
2,15123,1
3,15124,1
4,15125,1
